In [1]:
!git clone https://github.com/VuLamAnh151203/LLM2Rec_ex.git

Cloning into 'LLM2Rec_ex'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 107 (delta 46), reused 93 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 74.35 KiB | 6.20 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
%cd /kaggle/working/LLM2Rec_ex

/kaggle/working/LLM2Rec_ex


In [3]:
!pip install fire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.2 MB/s eta 0:00:00


In [4]:
import os

# Required to avoid prompt
os.environ["WANDB_API_KEY"] = "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"
os.environ["WANDB_MODE"] = "online"   # or "offline"

# Optional but recommended
os.environ["WANDB_DIR"] = "/kaggle/output/wandb"
os.environ["WANDB_SILENT"] = "true"

import wandb

wandb.init(
    project="LLM2Rec",
    name="kaggle_run",
)


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [5]:
# Stage 1: Collaborative Supervised Fine-Tuning (CSFT) for CiteULike

# Use a local path or HuggingFace ID
# model_path="Qwen/Qwen2-0.5B" 

# Relative path to the processed data we just created
# Assumes this script is run from LLM2Rec directory
train_file= "/kaggle/input/llm2rec-data/CiteULike/llm2rec_processed/train.csv"
eval_file="/kaggle/input/llm2rec-data/CiteULike/llm2rec_processed/valid.csv"
output_dir="./output/Qwen2-0.5B-CSFT-CiteULike"

!echo "Training with: ${train_file}"

# Run training
# Adjust nproc_per_node based on your GPU count
!torchrun --nproc_per_node 2 \
    ./llm2rec/run_csft.py \
    --base_model "Qwen/Qwen2-0.5B" \
    --train_file "{train_file}" \
    --eval_file "{eval_file}" \
    --output_dir "{output_dir}" \
    --wandb_run_name "Qwen2-0.5B-CSFT-CiteULike" \
    --category "CiteULike" \
    --train_from_scratch False \
    --use_lora True \
    --learning_rate 6e-5 \
    --num_epochs 10 \
    --batch_size 8 \
    --micro_batch_size 4

# Copy tokenizer to output for next stages
!echo "Copying tokenizer files..."
# Note: If model_path is a local directory, copy from there. 
# If it's a Hub ID, the tokenizer is saved by the script in output_dir mostly, but let's be safe.
# The script `run_csft.py` saves the tokenizer/model at the end.


Training with: $/kaggle/input/llm2rec-data/CiteULike/llm2rec_processed/train.csv
W0113 20:22:04.813000 86 torch/distributed/run.py:774] 
W0113 20:22:04.813000 86 torch/distributed/run.py:774] *****************************************
W0113 20:22:04.813000 86 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0113 20:22:04.813000 86 torch/distributed/run.py:774] *****************************************
2026-01-13 20:22:19.496540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-13 20:22:19.496540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already 

In [6]:
# !git pull